Now that we have gathered data from Basketball Reference using web scraping, cleaned our data, and conducted some exploratory data analysis, it's time to create our machine learning model to predict who will win the NBA MVP!

In [1]:
#Import Libraries
import pandas as pd
import numpy as np
#Machine Learning 
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, KFold, cross_val_predict, cross_val_score, ParameterGrid
from sklearn.metrics import mean_absolute_error, mean_squared_error, make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.inspection import permutation_importance
from sklearn.svm import SVR
#Ignore Warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Read in data
df = pd.read_csv('all_stats.csv')

In [3]:
df.head(10)

,Unnamed: 0,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,PF,PTS,Year,Pts Won,Pts Max,Share,Team,Wins,Losses,W%
0,0,A.C. Green,PF,23,LAL,79,72,28.4,4.0,7.4,...,2.2,10.8,1987,0.0,780.0,0.00,Los Angeles Lakers,65,17,0.79268
1,1,Adrian Branch,SF,23,LAL,32,0,6.8,1.5,3.0,...,1.2,4.3,1987,0.0,780.0,0.00,Los Angeles Lakers,65,17,0.79268
2,2,Billy Thompson,SF,23,LAL,59,0,12.9,2.4,4.4,...,2.5,5.6,1987,0.0,780.0,0.00,Los Angeles Lakers,65,17,0.79268
3,3,Byron Scott,SG,25,LAL,82,82,33.3,6.8,13.8,...,2.0,17.0,1987,0.0,780.0,0.00,Los Angeles Lakers,65,17,0.79268
4,4,James Worthy,SF,25,LAL,82,82,34.4,7.9,14.7,...,2.5,19.4,1987,0.0,780.0,0.00,Los Angeles Lakers,65,17,0.79268
5,5,Kareem Abdul-Jabbar,C,39,LAL,78,78,31.3,7.2,12.7,...,3.1,17.5,1987,0.0,780.0,0.00,Los Angeles Lakers,65,17,0.79268
6,6,Kurt Rambis,PF,28,LAL,78,10,19.4,2.1,4.0,...,2.6,5.7,1987,0.0,780.0,0.00,Los Angeles Lakers,65,17,0.79268
7,7,Magic Johnson,PG,27,LAL,80,80,36.3,8.5,16.4,...,2.1,23.9,1987,733.0,780.0,0.94,Los Angeles Lakers,65,17,0.79268
8,8,Michael Cooper,SG,30,LAL,82,2,27.5,3.9,9.0,...,2.4,10.5,1987,0.0,780.0,0.00,Los Angeles Lakers,65,17,0.79268
9,9,Mike Smrek,C,24,LAL,35,3,6.7,0.9,1.7,...,2.0,2.2,1987,0.0,780.0,0.00,Los Angeles Lakers,65,17,0.79268


In [4]:
#Delete Unwanted Column
del df['Unnamed: 0']

# Missing Data

In [5]:
df.isna().sum().sort_values(ascending=False)

3P%        2333
FT%         574
2P%         110
eFG%         64
FG%          64
Player        0
PF            0
AST           0
STL           0
BLK           0
TOV           0
PTS           0
DRB           0
Year          0
Pts Won       0
Pts Max       0
Share         0
Team          0
Wins          0
Losses        0
TRB           0
FTA           0
ORB           0
Pos           0
FT            0
2PA           0
2P            0
3PA           0
3P            0
FGA           0
FG            0
MP            0
GS            0
G             0
Tm            0
Age           0
W%            0
dtype: int64

It does appear that we do have some missing data. My assumption is that the missing data is for players who did not attempt a shot in the given category. For example, if a player never shot a 3, then their 3P% would be missing. Lets check this. 

In [6]:
#Define dataframe with null 3P%
null_3P = df[df['3P%'].isnull()]
#Show 3PA Data
null_3P['3PA'].value_counts()

3PA
0.0    2333
Name: count, dtype: int64

Looks like everyone who has missing data for 3P% also has 0 3PA. Thus, it does appear as those with null values never attempted a shot. Now we can explore the other categories with missing data. It is likely the same reasoning as 3P% applies. 

In [7]:
#Dictionary with the null values and attempts
null_values = {
    'FT%': 'FTA',
    '2P%': '2PA',
    'eFG%': 'FGA',
    'FG%': 'FGA',
}

for null_key, attempt_key in null_values.items():
    null_rows = df[df[null_key].isnull()]
    value_counts = null_rows[attempt_key].value_counts()
    print(f'Null Value: {null_key}')
    print(value_counts)
    print()

Null Value: FT%
FTA
0.0    574
Name: count, dtype: int64

Null Value: 2P%
2PA
0.0    110
Name: count, dtype: int64

Null Value: eFG%
FGA
0.0    64
Name: count, dtype: int64

Null Value: FG%
FGA
0.0    64
Name: count, dtype: int64



Looks like for all of the columns with missing data it is because the player has never taken that type of shot. To fill the missing values I will use the median for that position that year. We could also drop the rows with missing values; however, there are rare cases where players who do not attempt 3s are in the MVP running. For example, Shaquille O'Neal has had multiple season where he attempted 0 3 point shots but won the MVP in the 1999-2000 season. 

In [8]:
columns_to_fill = ['3P%', 'FT%', '2P%', 'eFG%', 'FG%']
for column in columns_to_fill:
    median_by_position_year = df.groupby(['Year', 'Pos'])[column].transform('median') 
    df[column].fillna(median_by_position_year, inplace=True)  

In [9]:
df.isna().sum().sort_values(ascending=False)

3P%        14
FT%         1
Player      0
PF          0
DRB         0
TRB         0
AST         0
STL         0
BLK         0
TOV         0
PTS         0
Year        0
Pts Won     0
Pts Max     0
Share       0
Team        0
Wins        0
Losses      0
ORB         0
FTA         0
Pos         0
FGA         0
Age         0
Tm          0
G           0
GS          0
MP          0
FG          0
FG%         0
FT          0
3P          0
3PA         0
2P          0
2PA         0
2P%         0
eFG%        0
W%          0
dtype: int64

There do still appear to be some missing values. For this I will use the median for the overall position

In [10]:
columns_to_fill = ['3P%', 'FT%', '2P%', 'eFG%', 'FG%']
for column in columns_to_fill:
    median_by_position = df.groupby('Pos')[column].transform('median')  
    df[column].fillna(median_by_position, inplace=True) 

In [11]:
df.isna().sum().sort_values(ascending=False)

Player     0
FT%        0
DRB        0
TRB        0
AST        0
STL        0
BLK        0
TOV        0
PF         0
PTS        0
Year       0
Pts Won    0
Pts Max    0
Share      0
Team       0
Wins       0
Losses     0
ORB        0
FTA        0
Pos        0
FT         0
Age        0
Tm         0
G          0
GS         0
MP         0
FG         0
FGA        0
FG%        0
3P         0
3PA        0
3P%        0
2P         0
2PA        0
2P%        0
eFG%       0
W%         0
dtype: int64

No more missing data, nice! Now we need to use dummy variables for the Position and Team columns

# Feature Engineering

In [12]:
# Create dummy variables for the Pos column
position_dummies = pd.get_dummies(df['Pos'],)
team_dummies = pd.get_dummies(df['Team'])
df_with_dummies = pd.concat([df, position_dummies, team_dummies], axis=1)
df_with_dummies.drop('Pos', axis=1, inplace=True)
df_with_dummies.drop('Team', axis=1, inplace=True)

We will now add features that meassure if a player scored above average for the year in the points, rebounds, assists, blocks, and steals categories. 

In [13]:
average_stats_for_year = df_with_dummies.groupby('Year')[['PTS', 'ORB', 'DRB', 'TRB', 'STL', 'AST', 'BLK']].transform('mean')
# Calculate the new columns by dividing player's statistics by average statistics for their year
statistics_columns = ['PTS', 'ORB', 'DRB', 'TRB', 'STL', 'AST', 'BLK']
for col in statistics_columns:
    df_with_dummies[f'{col}_C'] = df_with_dummies[col] / average_stats_for_year[col]

In [14]:
df_with_dummies.columns

Index(['Player', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA',
       '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB',
       'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year', 'Pts Won',
       'Pts Max', 'Share', 'Wins', 'Losses', 'W%', 'C', 'C-PF', 'PF', 'PF-C',
       'PF-SF', 'PG', 'PG-SF', 'PG-SG', 'SF', 'SF-C', 'SF-PF', 'SF-SG', 'SG',
       'SG-PF', 'SG-PG', 'SG-PG-SF', 'SG-SF', 'Atlanta Hawks',
       'Boston Celtics', 'Brooklyn Nets', 'Charlotte Bobcats',
       'Charlotte Hornets', 'Chicago Bulls', 'Cleveland Cavaliers',
       'Dallas Mavericks', 'Denver Nuggets', 'Detroit Pistons',
       'Golden State Warriors', 'Houston Rockets', 'Indiana Pacers',
       'Los Angeles Clippers', 'Los Angeles Lakers', 'Memphis Grizzlies',
       'Miami Heat', 'Milwaukee Bucks', 'Minnesota Timberwolves',
       'New Jersey Nets', 'New Orleans Hornets', 'New Orleans Pelicans',
       'New Orleans/Oklahoma City Hornets', 'New York Knicks',
       'Okl

In [15]:
features_full = ['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year',
       'Wins', 'Losses', 'W%', 'C', 'C-PF', 'PF', 'PF-C',
       'PF-SF', 'PG', 'PG-SF', 'PG-SG', 'SF', 'SF-C', 'SF-PF', 'SF-SG', 'SG',
       'SG-PF', 'SG-PG', 'SG-PG-SF', 'SG-SF', 'Atlanta Hawks',
       'Boston Celtics', 'Brooklyn Nets', 'Charlotte Bobcats',
       'Charlotte Hornets', 'Chicago Bulls', 'Cleveland Cavaliers',
       'Dallas Mavericks', 'Denver Nuggets', 'Detroit Pistons',
       'Golden State Warriors', 'Houston Rockets', 'Indiana Pacers',
       'Los Angeles Clippers', 'Los Angeles Lakers', 'Memphis Grizzlies',
       'Miami Heat', 'Milwaukee Bucks', 'Minnesota Timberwolves',
       'New Jersey Nets', 'New Orleans Hornets', 'New Orleans Pelicans',
       'New Orleans/Oklahoma City Hornets', 'New York Knicks',
       'Oklahoma City Thunder', 'Orlando Magic', 'Philadelphia 76ers',
       'Phoenix Suns', 'Portland Trail Blazers', 'Sacramento Kings',
       'San Antonio Spurs', 'Seattle SuperSonics', 'Toronto Raptors',
       'Utah Jazz', 'Vancouver Grizzlies', 'Washington Bullets',
       'Washington Wizards','PTS_C', 'ORB_C', 'DRB_C', 'TRB_C', 'STL_C',
       'AST_C', 'BLK_C']

Now we need to define our train and test data. We will use the data from 1987-2018 as the training data and then test on the 2019, 20, 21, and 22 seasons. We will leave the 2023 season unseen until the end so we can see how well our model does on data it has never seen.

In [16]:
#Training vs Testing
train = df_with_dummies[df_with_dummies['Year'] <= 2018]
test = df_with_dummies[(df_with_dummies['Year'] >= 2019) & (df_with_dummies['Year'] <= 2022)]

In [17]:
#Select features
VIF_df = df_with_dummies[features_full]

#Convert to integers
VIF_df = VIF_df.astype(int)

#Drop NANs
VIF_df = VIF_df.dropna()

#Calculate VIF 
vif_data = pd.DataFrame()
vif_data["Feature"] = VIF_df.columns
vif_data["VIF"] = [variance_inflation_factor(VIF_df.values, i) for i in range(VIF_df.shape[1])]

print(vif_data.to_string())

                              Feature         VIF
0                                 Age    1.153860
1                                   G    2.395401
2                                  GS    3.923618
3                                  MP   15.361448
4                                  FG   63.526685
5                                 FGA  132.055370
6                                 FG%   53.176632
7                                  3P    5.801088
8                                 3PA   21.916655
9                                 3P%    1.022360
10                                 2P   45.291877
11                                2PA   94.862645
12                                2P%    2.090589
13                               eFG%   52.113180
14                                 FT   14.312342
15                                FTA   14.278360
16                                FT%    1.104156
17                                ORB   10.006120
18                                DRB   20.172663


The high VIF are a concern. I will do forward stepwise selection with AIC as the decision criteria to select features. This will be done in R.

In [18]:
#Export data
data_export = df_with_dummies
data_export = data_export.drop('Player', axis=1)
data_export = data_export.drop('Tm', axis = 1)

data_export.to_csv("mvpData.csv")

The variables selected from AIC and forward stepwise selection are defined below

In [19]:
features = ['FTA', 'W%', 'BLK', 'MP', 'PTS_C', 'DRB_C', 'FGA', 'STL', 'FG%', 'TRB_C', 'TOV', 'PG', 'FT%',
           'G', 'C', 'ORB', '2PA', '2P', '2P%', 'Age', 'PF', 'eFG%', '3P%', 'AST_C']

In [20]:
X_train = train[features]
y_train = train['Share']
X_test = test[features]
y_test = test['Share']

# Model

Lets start by trying a simple Linear Regression model before finding a model that works best.

In [21]:
#Linear Regression
LinReg = LinearRegression()
LinReg.fit(X_train, y_train)

LinearRegression()

In [22]:
pred = LinReg.predict(X_test)

In [23]:
pred = pd.DataFrame(pred, columns=['Predicted Share'], index=test.index)

In [24]:
pred.head(10)

,Predicted Share
705,0.010955
706,0.002540
707,0.008581
708,0.000019
709,-0.014609
710,0.003249
711,-0.005602
712,-0.009876
713,0.004268
714,0.013384


In [25]:
results = pd.concat([test[['Player', 'Share', 'Year']], pred], axis=1)

In [26]:
results.sort_values('Share', ascending=False).head(10)

,Player,Share,Year,Predicted Share
751,Nikola Jokić,0.961,2021,0.149715
14564,Giannis Antetokounmpo,0.952,2020,0.226609
13469,Giannis Antetokounmpo,0.932,2019,0.213868
773,Nikola Jokić,0.875,2022,0.216613
11129,James Harden,0.768,2019,0.186719
4304,LeBron James,0.746,2020,0.154032
963,Joel Embiid,0.706,2022,0.195015
13169,Giannis Antetokounmpo,0.595,2022,0.239942
10014,Joel Embiid,0.580,2021,0.142084
4235,Stephen Curry,0.449,2021,0.127539


We need a way to meassure if this model did a good job so we can compare the performance of multiple models. We want to create a model that does a good job of predicting the Share of the best players so we will take this into consideration when creating an error metric. To do this we need to rank the players based on Share and Predicted Share. Lets just look at our predictions for the 2019 season.

In [27]:
results_2019 = results[results['Year'] == 2019]
results_2019 = results_2019.sort_values('Share', ascending=False)
results_2019['Rank'] = results_2019['Share'].rank(ascending=False).astype(int)
results_2019.head(6)

,Player,Share,Year,Predicted Share,Rank
13469,Giannis Antetokounmpo,0.932,2019,0.213868,1
11129,James Harden,0.768,2019,0.186719,2
1631,Paul George,0.352,2019,0.105687,3
13826,Nikola Jokić,0.210,2019,0.110594,4
4862,Stephen Curry,0.173,2019,0.099701,5
3441,Damian Lillard,0.068,2019,0.098504,6


In [28]:
results_2019 = results_2019.sort_values('Predicted Share', ascending=False)
results_2019['Predicted Rank'] = results_2019['Predicted Share'].rank(ascending=False).astype(int)
results_2019.sort_values('Share', ascending=False).head(6)

,Player,Share,Year,Predicted Share,Rank,Predicted Rank
13469,Giannis Antetokounmpo,0.932,2019,0.213868,1,1
11129,James Harden,0.768,2019,0.186719,2,2
1631,Paul George,0.352,2019,0.105687,3,10
13826,Nikola Jokić,0.210,2019,0.110594,4,9
4862,Stephen Curry,0.173,2019,0.099701,5,11
3441,Damian Lillard,0.068,2019,0.098504,6,12


It looks like the model did a decent job at predicting the MVP. We will need to look further into Anthony Davis as he did not recieve any MVP votes but our model predicted he finished fourth in voting.

In [29]:
results_2019['Difference'] = results_2019['Rank'] - results_2019['Predicted Rank']

In [30]:
results_2019[results_2019['Rank'] <= 6].sort_values('Difference', ascending=True)

,Player,Share,Year,Predicted Share,Rank,Predicted Rank,Difference
1631,Paul George,0.352,2019,0.105687,3,10,-7
4862,Stephen Curry,0.173,2019,0.099701,5,11,-6
3441,Damian Lillard,0.068,2019,0.098504,6,12,-6
13826,Nikola Jokić,0.210,2019,0.110594,4,9,-5
13469,Giannis Antetokounmpo,0.932,2019,0.213868,1,1,0
11129,James Harden,0.768,2019,0.186719,2,2,0


In [31]:
def ranks(df):
    df = df.sort_values('Share', ascending=False)
    df['Rank'] = df['Share'].rank(ascending=False).astype(int)
    df = df.sort_values('Predicted Share', ascending=False)  
    df['Predicted Rank'] = df['Predicted Share'].rank(ascending=False).astype(int)
    df['Difference'] = df['Rank'] - df['Predicted Rank']
    
    return df

# Cross-Validation

In [32]:
#Remove 1999 season
df_with_dummies = df_with_dummies[df_with_dummies['Year'] != 1999]

#Reset the index
df_with_dummies = df_with_dummies.reset_index(drop=True)

#Sort data by year
df_sorted = df_with_dummies.sort_values(by='Year')

#Store years
years = np.unique(df_sorted['Year'].values)

#Format data
X = df_sorted[features]
y = df_sorted['Share']

#Create scaled data for models
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [33]:
#Define MSE error metrics
mse = make_scorer(mean_squared_error)

In [34]:
#List to store results
results = []

In [35]:
#Define models to test (models that compile faster for testing)
models = [
    {
        #Linear regression
        'name': 'Linear Regression',
        'model': LinearRegression(),
        'parameters': {}
    },
    {
        #Ridge regression
        'name': 'Ridge Regression',
        'model': Ridge(),
        'parameters': {'alpha': np.logspace(-4, 2, 100)}
    },
    {
        #Lasso regression
        'name': 'Lasso Regression',
        'model': Lasso(),
        'parameters': {'alpha': np.logspace(-4, 2, 100)}
    },
    {
        #KNN
        'name': 'KNN',
        'model': KNeighborsRegressor(),
        'parameters': {'n_neighbors': range(1,51)}}
]

In [36]:
#Define models to test (List of all models tested)
models = [
    {
        #Linear regression
        'name': 'Linear Regression',
        'model': LinearRegression(),
        'parameters': {}
    },
    {
        #Ridge regression
        'name': 'Ridge Regression',
        'model': Ridge(),
        'parameters': {'alpha': np.logspace(-4, 2, num=100)}
    },
    {
        #Lasso regression
        'name': 'Lasso Regression',
        'model': Lasso(),
        'parameters': {'alpha': np.logspace(-4, 2, num=100)}
    },
    {
        #KNN
        'name': 'KNN',
        'model': KNeighborsRegressor(),
        'parameters': {'n_neighbors': range(1,51)}
    },
    {
        #Random Forest
        'name': 'Random Forest',
        'model': RandomForestRegressor(),
        'parameters': {
            'n_estimators': [500],
            'max_features': ['sqrt', 'log2', 24, 3]
        }
    },
    {
        #XGBoosting
        'name': 'XGBoost',
        'model': XGBRegressor(),
        'parameters': {
            'learning_rate': [0.01, 0.1, 0.2], 
            'n_estimators': [100, 200, 300]
        }
    },
    {
        #Support Vector Regression
        'name': 'SVR',
        'model': SVR(),
        'parameters': {
            'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
        }
    }
]

In [37]:
#Perform cross-validation

#List to store results
results_list = []

#Try each model
for model_data in models:
    #Save info
    model_name = model_data['name']
    model = model_data['model']
    params = model_data['parameters']
    
    #Print status
    print(f"{model_name} starting")
    
    #Check if model has parameters to tune
    if params:
        #Set initial MSE
        best_mse = float('inf')
        #Set initial best parameters
        best_params = None
        
        #Test different parameters
        for param in ParameterGrid(params):
            #Progress tracking
            print(param)
            
            #MSE for parameters
            param_mse = []
            
            #Use CV to find optimal parameters
            for year in years:
                #Get indices for training set
                train_ind = np.where(df_sorted['Year'].values != year)[0]
        
                #Get indices for test set
                test_ind = np.where(df_sorted['Year'].values == year)[0]
                
                #Make a copy of the test set
                df_test = df_sorted.iloc[test_ind].reset_index(drop = True)
        
                #Split data
                X_train, X_test = X_scaled[train_ind], X_scaled[test_ind]
                y_train, y_test = y[train_ind], y[test_ind]
                
                #Set parameters
                model.set_params(**param)
                
                #Fit model
                model.fit(X_train, y_train)
                
                #Make predictions
                y_pred = model.predict(X_test)
                
                #Save predictions
                df_test['Predicted Share'] = y_pred
                
                #Subset to only players who recieved votes
                df_test = df_test[df_test['Share'] != 0]
                
                #Set ranks
                df_test = ranks(df_test)
                
                #Calculate and save MSE on ranks
                mse_year = mean_squared_error(df_test['Rank'], df_test['Predicted Rank'])
                param_mse.append(mse_year)
                
            #Calculate MSE
            mse = np.mean(param_mse)
            
            #Update best model if needed
            if mse < best_mse:
                best_mse = mse
                best_params = param
                
        #Save results
        results_data = {'Model': model_name, 'MSE': best_mse, 'Best_Params': best_params}
        results_list.append(results_data)
            
    #Fit models that don't require parameter tuning
    else:
        #List to store MSEs
        mse_list = []
        
        #Loop over each year
        for year in years:
            #Get indices for training set
            train_ind = np.where(df_sorted['Year'].values != year)[0]
        
            #Get indices for test set
            test_ind = np.where(df_sorted['Year'].values == year)[0]
            
            #Make a copy of the test set
            df_test = df_sorted.iloc[test_ind].reset_index(drop = True)
        
            #Split data
            X_train, X_test = X_scaled[train_ind], X_scaled[test_ind]
            y_train, y_test = y[train_ind], y[test_ind]
            
            #Fit model
            model.fit(X_train, y_train)
            
            #Make predictions
            y_pred = model.predict(X_test) 
                
            #Save predictions
            df_test['Predicted Share'] = y_pred
                
            #Subset to only players who recieved votes
            df_test = df_test[df_test['Share'] != 0]
                
            #Set ranks
            df_test = ranks(df_test)
                
            #Calculate and save MSE on ranks
            mse_year = mean_squared_error(df_test['Rank'], df_test['Predicted Rank'])
            mse_list.append(mse_year)
            
        #Calculate MSE
        best_mse = np.mean(mse_list)
        
        #Save model results
        results_data = {'Model': model_name, 'MSE': best_mse}
        results_list.append(results_data)
    
    #Print status
    print(f"{model_name} done")

Linear Regression starting
Linear Regression done
Ridge Regression starting
{'alpha': 10000000000.0}
{'alpha': 7564633275.54629}
{'alpha': 5722367659.35022}
{'alpha': 4328761281.083061}
{'alpha': 3274549162.877732}
{'alpha': 2477076355.991714}
{'alpha': 1873817422.8603868}
{'alpha': 1417474162.9268076}
{'alpha': 1072267222.0103253}
{'alpha': 811130830.789689}
{'alpha': 613590727.3413163}
{'alpha': 464158883.3612773}
{'alpha': 351119173.4215127}
{'alpha': 265608778.2946684}
{'alpha': 200923300.2565046}
{'alpha': 151991108.2952933}
{'alpha': 114975699.53977357}
{'alpha': 86974900.26177834}
{'alpha': 65793322.465756826}
{'alpha': 49770235.64332114}
{'alpha': 37649358.067924716}
{'alpha': 28480358.68435805}
{'alpha': 21544346.900318824}
{'alpha': 16297508.346206434}
{'alpha': 12328467.39442066}
{'alpha': 9326033.468832199}
{'alpha': 7054802.310718645}
{'alpha': 5336699.231206302}
{'alpha': 4037017.2585965497}
{'alpha': 3053855.5088334125}
{'alpha': 2310129.700083158}
{'alpha': 1747528.4000

In [38]:
results_df = pd.DataFrame(results_list)
results_df.sort_values(by='MSE', ascending = True)

,Model,MSE,Best_Params
2,Lasso Regression,18.843813,{'alpha': 10000000000.0}
3,KNN,20.931174,{'n_neighbors': 1}
1,Ridge Regression,31.560110,{'alpha': 174.7528400007683}
4,Random Forest,32.501809,"{'max_features': 24, 'n_estimators': 500}"
0,Linear Regression,32.525656,NaN
6,SVR,33.498623,{'kernel': 'linear'}
5,XGBoost,34.055099,"{'learning_rate': 0.1, 'n_estimators': 300}"
